<a href="https://colab.research.google.com/github/sergekamanzi/ChatBotpro/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --upgrade datasets

In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# Complete Rwanda History Chatbot with Gradio
# ─────────────────────────────────────────────────────────────────────────────

# 1. Install dependencies
#    Run this cell first in Colab or your environment
!pip install -q gradio sentence-transformers

# 2. Imports
import re
import numpy as np
import gradio as gr
from sentence_transformers import SentenceTransformer, util

# 3. Full Rwanda history text
rwanda_history = """
Human occupation of Rwanda is thought to have begun shortly after the last ice age. By the 11th century, the inhabitants had organized into a number of kingdoms. In the 19th century, Mwami Rwabugiri of the Kingdom of Rwanda conducted military conquests and administrative consolidation to control most of present-day Rwanda. Germany and Belgium then allied with the Rwandan court.

In 1962 independence was granted; Grégoire Kayibanda became president. Unsettled ethnic tensions worsened under Juvénal Habyarimana's 1973 coup. In 1990 the Tutsi-led Rwandan Patriotic Front (RPF) invaded, triggering the Civil War. The assassination of Habyarimana in 1994 sparked the genocide: over ~100 days, 500,000–1,000,000 Tutsis and moderate Hutus were killed by Hutu militias incited on radio. The RPF under Paul Kagame ended the genocide by taking Kigali in July 1994, leading to a mass exodus of roughly two million Hutu refugees into neighboring countries.

Prehistoric & early ages:
Humans entered post–last ice age (~10,000 BCE) or up to 3000 BCE. The Twa Pygmy forest hunters were first, followed by Iron Age settlers with pottery and iron tools. Central Sudanic farmers arrived by 3000 BCE, Mashariki Bantu by 1000 BCE, and Proto–Great Lakes Bantu by 500 BCE, assimilating earlier Cushitic & Nilo-Saharan groups.

Kingdom era & Rwabugiri:
By the 15th century, small Hutu & Tutsi states existed (e.g., Gisaka). Under King Rwabugiri (19th century), the realm centralized—Tutsi chiefs collected tribute; the Mwami held semi-divine status. The Gacaca justice system administered local disputes.

Colonial period:
Germany claimed Rwanda in 1897; Belgium took over in 1916 under a League of Nations mandate. Belgians reinforced Tutsi dominance via cattle-based identity cards, introduced cash taxes, coffee cultivation, and forced labor. Ethnic quotas and identity cards (from 1931) hardened divisions. Famines in 1928–29 & 1943–44 killed up to one-third of the population.

Road to Independence:
Pan-Africanism & Hutu emancipation grew post-WWII. The “Hutu Manifesto” (1957) spurred conflict, the 1959 “wind of destruction” massacred 20,000–100,000 Tutsi, and Tutsi fled. A 1961 referendum abolished the monarchy; full independence came July 1, 1962.

Post-genocide & today:
Post-1994, Kagame’s RPF formed a unity government; Pasteur Bizimungu served as a Hutu president until 2000. Vision 2020 aims to build a knowledge-based economy. Today Rwanda is among Africa’s least corrupt nations but faces critiques on press freedom and political rights.
"""

# 4. Prepare sentence-level embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Split into sentences
sentences = re.split(r'\.\s*(?=[A-Z])', rwanda_history.strip())
sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

# 5. Define retrieval + response function
def find_relevant_info(question: str) -> str:
    question = question.strip()
    if not question:
        return "👋 Please ask a question about Rwanda’s history."

    # Encode and compute similarities
    q_emb = model.encode(question, convert_to_tensor=True)
    cos_scores = util.cos_sim(q_emb, sentence_embeddings)[0]
    top_indices = np.argsort(cos_scores.cpu().numpy())[-3:][::-1]

    # Filter by relevance threshold
    relevant = [sentences[i] for i in top_indices if cos_scores[i] > 0.3]
    if not relevant:
        return "I couldn't find specific info—try rephrasing or another topic."

    # Special handling for genocide queries
    if "genocide" in question.lower():
        return (
            "The 1994 Rwandan genocide was triggered by President Habyarimana’s assassination on April 6, 1994. "
            "Over ~100 days, 500,000–1,000,000 Tutsis and moderate Hutus were killed by Hutu militias incited via "
            "Radio Télévision Libre des Mille Collines. The RPF under Paul Kagame ended the genocide by capturing Kigali, "
            "leading to ~2 million Hutu refugees."
        )

    # Otherwise, join the top relevant sentences
    return ". ".join(relevant) + "."

# 6. Gradio chatbot wrapper
def chatbot_response(user_input: str) -> str:
    return find_relevant_info(user_input)

# 7. Launch Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about Rwanda’s history..."),
    outputs="text",
    title="Rwanda History Chatbot",
    description="Ask about Rwanda's past—from prehistoric times through the genocide to modern development."
)

if __name__ == "__main__":
    interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee592af5750b2467dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
